In [1]:
import watershed_workflow.sources.manager_nlcd
import pandas as pd

In [2]:
nlcd_colors = watershed_workflow.sources.manager_nlcd.colors
nlcd_inds = dict((k,v[0]) for (k,v) in nlcd_colors.items() if k != 0 and k != 127)

In [3]:
lc = pd.DataFrame({'indices':nlcd_inds.keys(), 'names':nlcd_inds.values()})
lc

,indices,names
0,11,Open Water
1,12,Perrenial Ice/Snow
2,21,"Developed, Open Space"
3,22,"Developed, Low Intensity"
4,23,"Developed, Medium Intensity"
5,24,"Developed, High Intensity"
6,31,Barren Land
7,41,Deciduous Forest
8,42,Evergreen Forest
9,43,Mixed Forest


In [4]:
# PT coefficients may often need to be tuned!
lc['Priestley-Taylor alpha of snow [-]'] = 1.26
lc['Priestley-Taylor alpha of bare ground [-]'] = 1.26
lc['Priestley-Taylor alpha of canopy [-]'] = 1.26
lc['Priestley-Taylor alpha of transpiration [-]'] = 1.26

# shortwave is reasonable, longwave just absorbs it all
lc["Beer's law extinction coefficient, shortwave [-]"] = 0.6
lc["Beer's law extinction coefficient, longwave [-]"] = 5

# default to a reasonable value with little-to-no microtopography
lc["snow transition depth [m]"] = 0.02

# defaults for grass/no vei
lc['rooting depth max [m]'] = 100.

# by default we let the LAI take care of this rather than turn off deciduous 
# transpiration manually the way that PRMS does it. 
lc['leaf on time [doy]'] = -1
lc['leaf off time [doy]'] = -1


In [5]:
lc = lc.set_index('indices', drop=True)
lc

,names,Priestley-Taylor alpha of snow [-],Priestley-Taylor alpha of bare ground [-],Priestley-Taylor alpha of canopy [-],Priestley-Taylor alpha of transpiration [-],"Beer's law extinction coefficient, shortwave [-]","Beer's law extinction coefficient, longwave [-]",snow transition depth [m],rooting depth max [m],leaf on time [doy],leaf off time [doy]
indices,,,,,,,,,,,
11,Open Water,1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1
12,Perrenial Ice/Snow,1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1
21,"Developed, Open Space",1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1
22,"Developed, Low Intensity",1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1
23,"Developed, Medium Intensity",1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1
24,"Developed, High Intensity",1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1
31,Barren Land,1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1
41,Deciduous Forest,1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1
42,Evergreen Forest,1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1


In [6]:
# Rooting profiles from CLM4.5 technical note
lc['rooting profile alpha [-]'] = 11.0
lc['rooting profile beta [-]'] = 2.0

# 41: Deciduous Forest → BDT Temperate
lc.loc[41, "rooting profile alpha [-]"] = 6.0   # BDT Temperate
lc.loc[41, "rooting profile beta [-]"]  = 2.0   # BDT Temperate

# 42: Evergreen Forest → NET Temperate
lc.loc[42, "rooting profile alpha [-]"] = 7.0   # NET Temperate
lc.loc[42, "rooting profile beta [-]"]  = 2.0   # NET Temperate

# 43: Mixed Forest → NET Temperate (per your instruction)
lc.loc[43, "rooting profile alpha [-]"] = 7.0   # NET Temperate
lc.loc[43, "rooting profile beta [-]"]  = 2.0   # NET Temperate

# 51: Dwarf Scrub → BDS Boreal
lc.loc[51, "rooting profile alpha [-]"] = 7.0   # BDS Boreal
lc.loc[51, "rooting profile beta [-]"]  = 1.5   # BDS Boreal

# 52: Shrub/Scrub → BDS Temperate
lc.loc[52, "rooting profile alpha [-]"] = 7.0   # BDS Temperate
lc.loc[52, "rooting profile beta [-]"]  = 1.5   # BDS Temperate

# 82: Cultivated Crops → Crop (all crop PFTs share same values)
lc.loc[82, "rooting profile alpha [-]"] = 6.0   # Crop PFT
lc.loc[82, "rooting profile beta [-]"]  = 3.0   # Crop PFT

# 90: Woody Wetlands → BDT Temperate
lc.loc[90, "rooting profile alpha [-]"] = 6.0   # BDT Temperate
lc.loc[90, "rooting profile beta [-]"]  = 2.0   # BDT Temperate


In [7]:
# capillary pressure [Pa] at fully closed stomata (wilting point)
lc["maximum xylem capillary pressure [Pa]"] = 2240000

# 21: Developed Open Space → C3 grass
lc.loc[21, "maximum xylem capillary pressure [Pa]"] = 2750000   # C3 grass

# 22: Developed Low Intensity → C3 grass
lc.loc[22, "maximum xylem capillary pressure [Pa]"] = 2750000   # C3 grass

# 41: Deciduous Forest → BDT Temperate
lc.loc[41, "maximum xylem capillary pressure [Pa]"] = 2240000   # BDT Temperate

# 42: Evergreen Forest → NET Temperate
lc.loc[42, "maximum xylem capillary pressure [Pa]"] = 2550000   # NET Temperate

# 43: Mixed Forest → NET Temperate
lc.loc[43, "maximum xylem capillary pressure [Pa]"] = 2550000   # NET Temperate

# 51: Dwarf Scrub → BDS Boreal
lc.loc[51, "maximum xylem capillary pressure [Pa]"] = 4280000   # BDS Boreal

# 52: Shrub/Scrub → BDS Temperate
lc.loc[52, "maximum xylem capillary pressure [Pa]"] = 4280000   # BDS Temperate

# 71: Grassland/Herbaceous → C3/C4 grass
lc.loc[71, "maximum xylem capillary pressure [Pa]"] = 2750000   # C3/C4 grass

# 81: Pasture/Hay → C3 grass
lc.loc[81, "maximum xylem capillary pressure [Pa]"] = 2750000   # C3 grass

# 82: Crops → Crop PFT
lc.loc[82, "maximum xylem capillary pressure [Pa]"] = 2750000   # Crop PFT

# 90: Woody Wetlands → BDT Temperate
lc.loc[90, "maximum xylem capillary pressure [Pa]"] = 2240000   # BDT Temperate

# 95: Herbaceous Wetlands → C3 grass
lc.loc[95, "maximum xylem capillary pressure [Pa]"] = 2750000   # C3 grass

# Optional: Barren/impervious NLCD types as NaN
# lc.loc[[11, 23, 24, 31], "maximum xylem capillary pressure [Pa]"] = np.nan


In [8]:
lc["albedo of bare ground [-]"] = 0.2
lc["albedo of canopy [-]"] = 0.11
lc["emissivity of bare ground [-]"] = 0.95


# NLCD 11: Open Water
lc.loc[11, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.06, 0.06, 0.98]  # Open Water

# NLCD 21,22: Developed Open Space / Low Intensity → grass/turf
lc.loc[[21,22], ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.18, 0.18, 0.95]  # Grass/Turf

# NLCD 23,24: Developed Medium/High Intensity → impervious
lc.loc[[23,24], ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.15, 0.15, 0.95]  # Impervious

# NLCD 31: Barren Land
lc.loc[31, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.20, 0.20, 0.95]  # Bare soil

# NLCD 41: Deciduous Forest → BDT Temperate
lc.loc[41, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.15, 0.18, 0.95]

# NLCD 42: Evergreen Forest → NET Temperate
lc.loc[42, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.12, 0.10, 0.97]

# NLCD 43: Mixed Forest → NET Temperate
lc.loc[43, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.12, 0.12, 0.97]

# NLCD 51: Dwarf Scrub → BDS Boreal
lc.loc[51, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.18, 0.18, 0.95]

# NLCD 52: Shrub/Scrub → BDS Temperate
lc.loc[52, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.18, 0.18, 0.95]

# NLCD 71: Grassland/Herbaceous → C3/C4 grass
lc.loc[71, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.18, 0.18, 0.95]

# NLCD 81: Pasture/Hay → C3 grass
lc.loc[81, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.18, 0.18, 0.95]

# NLCD 82: Crops → Crop PFT
lc.loc[82, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.18, 0.20, 0.95]

# NLCD 90: Woody Wetlands → BDT Temperate
lc.loc[90, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.12, 0.15, 0.95]

# NLCD 95: Herbaceous Wetlands → C3 grass
lc.loc[95, ["albedo of bare ground [-]", "albedo of canopy [-]", "emissivity of bare ground [-]"]] = [0.15, 0.15, 0.95]


In [9]:
# Ensure the column exists
lc["Manning's n [-]"] = 0.05  # default value

# Assign by NLCD, with USACE guidance
lc.loc[11, "Manning's n [-]"] = 0.03     # Open Water (USACE, Levee SOP, https://mmc.sec.usace.army.mil/levees-sop.html)
lc.loc[21, "Manning's n [-]"] = 0.04     # Developed, Open Space (USACE)
lc.loc[22, "Manning's n [-]"] = 0.09     # Developed, Low Intensity (USACE)
lc.loc[23, "Manning's n [-]"] = 0.12     # Developed, Medium Intensity (USACE)
lc.loc[24, "Manning's n [-]"] = 0.16     # Developed, High Intensity (USACE)
lc.loc[31, "Manning's n [-]"] = 0.026    # Barren Land (USACE)
lc.loc[41, "Manning's n [-]"] = 0.15     # Deciduous Forest (USACE)
lc.loc[42, "Manning's n [-]"] = 0.12     # Evergreen Forest (USACE)
lc.loc[43, "Manning's n [-]"] = 0.14     # Mixed Forest (USACE)
lc.loc[51, "Manning's n [-]"] = 0.11     # Dwarf Scrub (approx shrub, USACE)
lc.loc[52, "Manning's n [-]"] = 0.11     # Shrub / Scrub (USACE)
lc.loc[71, "Manning's n [-]"] = 0.04     # Grassland / Herbaceous (USACE)
lc.loc[81, "Manning's n [-]"] = 0.04     # Pasture / Hay (USACE)
lc.loc[82, "Manning's n [-]"] = 0.03     # Cultivated Crops (USACE)
lc.loc[90, "Manning's n [-]"] = 0.10     # Woody Wetlands (USACE)
lc.loc[95, "Manning's n [-]"] = 0.07     # Herbaceous Wetlands (USACE)


In [10]:
lc

,names,Priestley-Taylor alpha of snow [-],Priestley-Taylor alpha of bare ground [-],Priestley-Taylor alpha of canopy [-],Priestley-Taylor alpha of transpiration [-],"Beer's law extinction coefficient, shortwave [-]","Beer's law extinction coefficient, longwave [-]",snow transition depth [m],rooting depth max [m],leaf on time [doy],leaf off time [doy],rooting profile alpha [-],rooting profile beta [-],maximum xylem capillary pressure [Pa],albedo of bare ground [-],albedo of canopy [-],emissivity of bare ground [-],Manning's n [-]
indices,,,,,,,,,,,,,,,,,,
11,Open Water,1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1,11.0,2.0,2240000,0.06,0.06,0.98,0.030
12,Perrenial Ice/Snow,1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1,11.0,2.0,2240000,0.20,0.11,0.95,0.050
21,"Developed, Open Space",1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1,11.0,2.0,2750000,0.18,0.18,0.95,0.040
22,"Developed, Low Intensity",1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1,11.0,2.0,2750000,0.18,0.18,0.95,0.090
23,"Developed, Medium Intensity",1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1,11.0,2.0,2240000,0.15,0.15,0.95,0.120
24,"Developed, High Intensity",1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1,11.0,2.0,2240000,0.15,0.15,0.95,0.160
31,Barren Land,1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1,11.0,2.0,2240000,0.20,0.20,0.95,0.026
41,Deciduous Forest,1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1,6.0,2.0,2240000,0.15,0.18,0.95,0.150
42,Evergreen Forest,1.26,1.26,1.26,1.26,0.6,5,0.02,100.0,-1,-1,7.0,2.0,2550000,0.12,0.10,0.97,0.120


In [12]:
lc.to_csv('../watershed_workflow/data/nlcd_land_cover_properties.csv')


In [13]:
# additionally, we want manning's n by stream order
data = {
    "order": [1, 2, 3, 4, 5],
    "Manning's n [-]":     [0.2, 0.1, 0.065, 0.045, 0.03],
}

mann = pd.DataFrame(data).set_index("order")
mann.to_csv('../watershed_workflow/data/manning_n_by_stream_order.csv')